In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import json
import datetime
import math

In [2]:
import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## Importing data

In [3]:
matches = pd.read_csv("../Data/matches.csv")
matches.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode
0,2015-03-04,AFG,AUS,Away,3773,65
1,2012-08-25,AFG,AUS,Neutral,3437,156
2,2019-06-01,AFG,AUS,Neutral,4306,149
3,2014-03-01,AFG,BAN,Away,3623,190
4,2016-09-25,AFG,BAN,Away,3936,198


In [4]:
attributes = pd.read_csv("../Data/attributes.csv")
attributes.head()

,Full Name,Born,Bats,Bowls,Code
0,David A Warner,1986-10-27,Left Handed,Right Arm Leg Break,3637
1,Aaron James Finch,1986-11-17,Right Handed,Left Arm Orthodox,3845
2,Steven Peter Devereux Smith,1989-02-06,Right Handed,Right Arm Leg Break,3756
3,Glenn James Maxwell,1988-10-14,Right Handed,Right Arm Off Break,3969
4,James Peter Faulkner,1990-04-29,Right Handed,Left Arm Medium,3930


In [5]:
batsmen = open('../Data/batsmen.json',) 
batsmen_data = json.load(batsmen)

In [6]:
bowlers = open('../Data/bowlers.json',) 
bowlers_data = json.load(bowlers)

In [7]:
invtmap = open('../Data/invtmap.json',) 
invtmap_data = json.load(invtmap)

In [8]:
scorecard = open('../Data/scorecard.json',) 
scorecard_data = json.load(scorecard)

In [9]:
region = open('../Data/region.json',) 
region_data = json.load(region)

In [10]:
tmap = open('../Data/tmap.json',) 
tmap_data = json.load(tmap)

## Making the Database

In [11]:
def get_matches(team_1,team_2,date,num_years=5):
    matches_team=matches[matches['Team_1']==team_1]
    matches_team=matches_team[matches['Team_2']==team_2]
    matches_team1=matches[matches['Team_1']==team_2]
    matches_team1=matches_team1[matches['Team_2']==team_1]
    matches_team = pd.concat([matches_team,matches_team1],axis=0)
    matches_team['Date']= pd.to_datetime(matches_team['Date'])
    get_date = datetime.datetime.strptime(date, '%Y-%m-%d')
    min_date = datetime.datetime.strptime(str(get_date.year-num_years)+'-01'+'-01', '%Y-%m-%d')
    matches_team_latest = matches_team[matches_team['Date']>=min_date]
    matches_team_latest = matches_team_latest[matches_team_latest['Date']<date]
    matches_team_latest = matches_team_latest.sort_values(by='Date')
    
    matches_team_latest = matches_team_latest.reset_index()
    matches_team_latest=matches_team_latest.drop(['index'],axis=1)
    return matches_team_latest

In [12]:
def create_Dataset(Dataset):
    Dataset['Toss'] = ''
    Dataset['Fours_team1'] = ''
    Dataset['Fours_team2'] = ''
    Dataset['Sixes_team1'] = ''
    Dataset['Sixes_team2'] = ''
    Dataset['Strike_rate_team1'] = ''
    Dataset['Strike_rate_team2'] = ''
    Dataset['Maidens_team1'] = ''
    Dataset['Maidens_team2'] = ''
    Dataset['Wickets_team1'] = ''
    Dataset['Wickets_team2'] = ''
    Dataset['Economy_rate_team1'] = ''
    Dataset['Economy_rate_team2'] = ''
    Dataset['Score_1']=''
    Dataset['Score_2']=''
    Dataset['Result'] = ''
    
    MatchCodes = Dataset['MatchCode'].tolist()
    
    for i in range(len(MatchCodes)):
        matchdata = scorecard_data[str(MatchCodes[i])]
        #Dataset = Dataset.drop(columns=['GroundCode','Team_1','Team_2'])
        if (matchdata['TOSS']==matchdata['ORDER'][0]):
            toss = 1
        else:
            toss = 2
        #Batting 1
        batting1 = matchdata['BATTING1']
        Runs_team1 = []
        Balls_Played_team1 = []
        Fours_team1 = []
        Sixes_team1 = []
        Strike_rate_team1 = []
        for val in range(10):
            Runs_team1.append(batting1[val][2])
            Balls_Played_team1.append(batting1[val][3])
            Fours_team1.append(batting1[val][4])
            Sixes_team1.append(batting1[val][5])
            Strike_rate_team1.append(batting1[val][6])

        #Batting 2
        batting2 = matchdata['BATTING2']
        Runs_team2 = []
        Balls_Played_team2 = []
        Fours_team2 = []
        Sixes_team2 = []
        Strike_rate_team2 = []
        for val in range(10):
            Runs_team2.append(batting2[val][2])
            Balls_Played_team2.append(batting2[val][3])
            Fours_team2.append(batting2[val][4])
            Sixes_team2.append(batting2[val][5])
            Strike_rate_team2.append(batting2[val][6])

        #Bowling 1
        bowling1 = matchdata['BOWLING1']
        Overs_team1 = []
        Maidens_team1 = []
        Runs_given_team1 = []
        Wickets_team1 = []
        Economy_rate_team1 = []
        for val in range(10):
            if val<len(bowling1):
                Overs_team1.append(bowling1[val][1])
                Maidens_team1.append(bowling1[val][2])
                Runs_given_team1.append(bowling1[val][3])
                Wickets_team1.append(bowling1[val][4])
                Economy_rate_team1.append(bowling1[val][5])
            else:
                Overs_team1.append(-1)
                Maidens_team1.append(-1)
                Runs_given_team1.append(-1)
                Wickets_team1.append(-1)
                Economy_rate_team1.append(-1)

        #Bowling 2
        bowling2 = matchdata['BOWLING2']
        Overs_team2 = []
        Maidens_team2 = []
        Runs_given_team2 = []
        Wickets_team2 = []
        Economy_rate_team2 = []
        for val in range(10):
            if val<len(bowling2):
                Overs_team2.append(bowling2[val][1])
                Maidens_team2.append(bowling2[val][2])
                Runs_given_team2.append(bowling2[val][3])
                Wickets_team2.append(bowling2[val][4])
                Economy_rate_team2.append(bowling2[val][5])
            else:
                Overs_team2.append(-1)
                Maidens_team2.append(-1)
                Runs_given_team2.append(-1)
                Wickets_team2.append(-1)
                Economy_rate_team2.append(-1)
        #Scores
        score = matchdata['SCORES']
        if (score[0]>score[1]):
            result = 1
        elif (score[1]>score[0]):
            result = 2
        else:
            result = 'Tie'

        #insert into dataset
        Dataset['Toss'].iloc[i] = toss
        Dataset['Score_1'].iloc[i]=score[0]
        Dataset['Score_2'].iloc[i]=score[1]
        Dataset['Fours_team1'].iloc[i] = Fours_team1
        Dataset['Fours_team2'].iloc[i] = Fours_team2
        Dataset['Sixes_team1'].iloc[i] = Sixes_team1
        Dataset['Sixes_team2'].iloc[i] = Sixes_team2
        Dataset['Strike_rate_team1'].iloc[i] = Strike_rate_team1
        Dataset['Strike_rate_team2'].iloc[i] = Strike_rate_team2
        Dataset['Maidens_team1'].iloc[i] = Maidens_team1
        Dataset['Maidens_team2'].iloc[i] = Maidens_team2
        Dataset['Wickets_team1'].iloc[i] = Wickets_team1
        Dataset['Wickets_team2'].iloc[i] = Wickets_team2
        Dataset['Economy_rate_team1'].iloc[i] = Economy_rate_team1
        Dataset['Economy_rate_team2'].iloc[i] = Economy_rate_team2
        Dataset['Result'].iloc[i] = result
    
    Dataset = Dataset.drop(columns=['Date','GroundCode'])
    return Dataset
    

In [14]:
#Example Input
team_1 = 'IND'
team_2 = 'ZIM' 
date = '2015-07-12'

In [15]:
Dataset=get_matches(team_1,team_2,date)
Dataset=create_Dataset(Dataset)

<ipython-input-11-29ebaa57a627>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matches_team=matches_team[matches['Team_2']==team_2]
<ipython-input-11-29ebaa57a627>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matches_team1=matches_team1[matches['Team_2']==team_1]
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [16]:
Dataset.head()

,Team_1,Team_2,Venue,MatchCode,Toss,Fours_team1,Fours_team2,Sixes_team1,Sixes_team2,Strike_rate_team1,Strike_rate_team2,Maidens_team1,Maidens_team2,Wickets_team1,Wickets_team2,Economy_rate_team1,Economy_rate_team2,Score_1,Score_2,Result
0,IND,ZIM,Away,3111,1,"[2.0, 1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1]","[5.0, 4.0, 0.0, 5.0, 0.0, 4.0, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1]","[1.0, 2.0, 0.0, 0.0, 3.0, 0.0, -1, -1, -1, -1]","[56.41, 45.83, 0.0, 95.8, 69.81, 100.0, 220.0,...","[106.98, 78.64, 62.79, 111.67, 128.0, 150.0, -...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1]","[6.3, 18.0, 5.3, 4.6, 4.0, 5.71, 5.0, -1, -1, -1]","[6.68, 6.38, 6.0, 4.7, 5.1, 7.0, 5.0, -1, -1, -1]",285,289,2
1,IND,ZIM,Away,3116,2,"[4.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, ...","[4.0, 6.0, 2.0, 1.0, 1.0, -1, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, ...","[2.0, 1.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[70.21, 37.5, 62.07, 52.0, 60.0, 70.83, 60.0, ...","[76.74, 82.22, 86.96, 106.67, 76.47, -1, -1, -...","[1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[1.0, 1.0, 3.0, 1.0, 1.0, -1, -1, -1, -1, -1]","[0.0, 1.0, 0.0, 0.0, 2.0, -1, -1, -1, -1, -1]","[2.2, 3.7, 4.5, 4.1, 4.2, -1, -1, -1, -1, -1]","[6.48, 3.5, 5.0, 6.22, 3.86, -1, -1, -1, -1, -1]",194,197,2
2,IND,ZIM,Away,3537,2,"[3.0, 6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1]","[2.0, 3.0, 13.0, 4.0, 0.0, 1.0, -1, -1, -1, -1]","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1, -1, -1, -1]","[47.22, 73.21, 51.72, 91.67, 80.0, 28.57, 126....","[50.0, 80.95, 106.48, 75.0, 0.0, 53.33, -1, -1...","[1.0, 1.0, 1.0, 3.0, 1.0, 0.0, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[1.0, 1.0, 1.0, 0.0, 3.0, 1.0, -1, -1, -1, -1]","[1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, -1, -1, -1]","[6.33, 4.5, 3.9, 3.3, 4.3, 2.0, -1, -1, -1, -1]","[5.0, 3.33, 6.0, 3.4, 8.3, 6.0, 6.0, -1, -1, -1]",228,230,2
3,IND,ZIM,Away,3539,2,"[0.0, 11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0,...","[7.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, ...","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, ...","[2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[14.29, 91.34, 77.78, 20.0, 36.36, 93.24, 68.1...","[84.62, 57.14, 72.34, 0.0, 45.45, 15.38, 79.31...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.0, 0.0, 3.0, 1.0, 1.0, 0.0, -1, -1, -1, -1]","[1.0, 2.0, 1.0, 0.0, 1.0, 0.0, 1.0, -1, -1, -1]","[0.0, 1.0, 4.0, 1.0, 2.0, 0.0, -1, -1, -1, -1]","[7.0, 3.38, 5.11, 5.0, 6.7, 3.75, 11.33, -1, -...","[5.44, 5.2, 4.1, 3.0, 4.6, 5.0, -1, -1, -1, -1]",294,236,1
4,IND,ZIM,Away,3541,2,"[0.0, 0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, ...","[2.0, 5.0, 5.0, 2.0, 3.0, -1, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[0.0, 33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26...","[66.67, 109.38, 77.27, 61.11, 155.56, -1, -1, ...","[0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, -1, -1, -1]","[0.0, 0.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[1.0, 2.0, 1.0, 0.0, 1.0, 0.0, 4.0, -1, -1, -1]","[1.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[4.57, 2.78, 3.43, 7.0, 3.9, 2.0, 4.7, -1, -1,...","[6.63, 5.14, 4.53, 4.1, 7.0, -1, -1, -1, -1, -1]",183,187,2


### Data for model

In [17]:
def get_model_data(Dataset):
    venue_encoding = {
    
    "Home" : 0,
    "Away" : 1,
    "Neutral" : 2
    }
    Dataset.replace({'Venue':venue_encoding}, inplace=True)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(enc.fit_transform(Dataset[['Result','Toss']]).toarray())
    Dataset = Dataset.join(enc_df)
    Dataset.rename(columns = {0: 'Win1',1: 'Win2',2: 'Toss1',3: 'Toss2'}, inplace = True)
    y = Dataset[['Result','Win1','Win2','Score_1','Score_2']]
    Dataset.drop(['Result','Score_1','Score_2'],inplace=True, axis=1)
    Dataset.drop(['Team_1','Team_2'],inplace=True, axis=1)
    temp=Dataset['Venue']
    Dataset.drop(['Win1','Win2'],inplace=True, axis=1)
    Dataset.drop(['Venue'],inplace=True, axis=1)
    Dataset['Venue']=''
    Dataset['Venue']=temp
    Dataset.drop(['Toss'],inplace=True, axis=1)
    Dataset['new']=''
    for i in range(len(Dataset)):
        l=[]
        for j in range(1,len(Dataset.columns)-4):
            if(type(Dataset.iloc[i][j])!=list):
                l.append(Dataset.iloc[i][j])
            else:
                for k in range(len(Dataset.iloc[i][j])):
                    l.append(Dataset.iloc[i][j][k])
        Dataset['new'].iloc[i]=l 
    
    
    X=Dataset[['MatchCode','new','Toss1','Toss2','Venue']]
    return X,y

In [18]:
X,y=get_model_data(Dataset)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [19]:
# Test Train Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/5, random_state = 100)

### Predict_data

In [20]:
def allBatsmen(batting):
    batting_keys=batting.keys()
    performance={}
    for k in batting_keys:
        for j in batting[k]:
            performance.setdefault(k,[]).append([j[0],j[1][1],j[1][2],j[1][3],j[1][4],j[1][5]])

    performance_keys = performance.keys()
    
    for per in performance_keys:
        l = performance[per]
        l1=sorted(l,reverse=True)
        performance[per]=l1
        
    return performance

In [21]:
def allBowlers(bowling):
    bowling_keys=bowling.keys()
    performance={}
    for k in bowling_keys:
        for j in bowling[k]:
            performance.setdefault(k,[]).append([j[0],j[1][0],j[1][1],j[1][2],j[1][3],j[1][4]])

    performance_keys = performance.keys()
    
    for per in performance_keys:
        l = performance[per]
        l1=sorted(l,reverse=True)
        performance[per]=l1
        
    return performance

In [22]:
def getLatestPerf_Bat(perform,team_players, date, num_years=5):
    latest_performance={}
    perform_keys = perform.keys()
    for k in team_players:
        for i in perform[k]:
            d = datetime.datetime.strptime(date, '%Y-%m-%d')
            d1 = datetime.datetime.strptime(i[0], '%Y-%m-%d')
            min_date = datetime.datetime.strptime(str(d.year-num_years)+'-01'+'-01', '%Y-%m-%d')
            if(d1>=min_date and i[0]<date):
                latest_performance.setdefault(k,[]).append(i)
    return latest_performance

In [23]:
def getAvgBat(team,index):
    final_score=[]
    for k in team.keys():
        para=[]
        for j in team[k]:
            para.append(j[index])
        
        para1=pd.Series(para)
        f1=para1.ewm(com=0.5,ignore_na=True,min_periods=1).mean()
       
        f1=np.nanmean(f1)
        final_score.append(round(f1,3))
            
    return final_score

In [24]:
def getLatestPerf_Bowl(perform,team_players, date, num_years=5):
    latest_performance={}
    perform_keys = perform.keys()
    for k in team_players:
        for i in perform[k]:
            d = datetime.datetime.strptime(date, '%Y-%m-%d')
            d1 = datetime.datetime.strptime(i[0], '%Y-%m-%d')
            min_date = datetime.datetime.strptime(str(d.year-num_years)+'-01'+'-01', '%Y-%m-%d')
            if(d1>=min_date and i[0]<date):
                latest_performance.setdefault(k,[]).append(i)
    return latest_performance

In [25]:
def getAvgBowl(team,index):
    final_score=[]
    for k in team.keys():
        para=[]
        for j in team[k]:
            para.append(j[index])
            
        para1=[x if x!=-1 else np.nan for x in para]
        para1=pd.Series(para1)
        f1=para1.ewm(com=0.5).mean()
        f1=np.nanmean(f1)
        final_score.append(round(f1,3))
    
    return final_score

In [26]:
def adjust_length(l):
    if (len(l)<10):
        l.extend([-1] * (10 - len(l)))
    return l

In [27]:
match_codes=X_test['MatchCode'].tolist()

In [28]:
bat={}
bat=allBatsmen(batsmen_data)
print(len(bat))

2099


In [29]:
bowl={}
bowl=allBowlers(bowlers_data)
print(len(bowl))

1616


In [30]:
for i in range(len(match_codes)):
    match_code=match_codes[i]
    scorecard=scorecard_data[str(match_code)]
    team_1_batsmen = [scorecard['BATTING1'][i][0] for i in range(10)]
    team_2_batsmen = [scorecard['BATTING2'][i][0] for i in range(10)]
    team_1_bowlers = [scorecard['BOWLING1'][i][0] for i in range(len(scorecard['BOWLING1']))]
    team_2_bowlers = [scorecard['BOWLING2'][i][0] for i in range(len(scorecard['BOWLING2']))]
    date=matches[matches['MatchCode']==match_code]['Date'].iloc[0]
    team1_bat=getLatestPerf_Bat(bat,team_1_batsmen, date, 5)
    team2_bat=getLatestPerf_Bat(bat,team_2_batsmen, date, 5)
    team1_bowl=getLatestPerf_Bowl(bowl,team_1_bowlers, date, 5)
    team2_bowl=getLatestPerf_Bowl(bowl,team_2_bowlers, date, 5)
    
    row = adjust_length(getAvgBat(team1_bat,3)) + adjust_length(getAvgBat(team2_bat,3))+adjust_length(getAvgBat(team1_bat,4))+\
    adjust_length(getAvgBat(team2_bat,4))+adjust_length(getAvgBat(team1_bat,5))+adjust_length(getAvgBat(team2_bat,5))+\
    adjust_length(getAvgBowl(team1_bowl,1))+adjust_length(getAvgBowl(team2_bowl,1))+adjust_length(getAvgBowl(team1_bowl,3))+\
    adjust_length(getAvgBowl(team2_bowl,3))+adjust_length(getAvgBowl(team1_bowl,4))+\
    adjust_length(getAvgBowl(team2_bowl,4))
    
    X_test['new'].iloc[i]=row
    #print(len(X_test['new'].iloc[i]))

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-30-2b40cc4e9323>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['new'].iloc[i]=row
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-

In [31]:
l=[]
X_train['all']=''
for i in range(len(X_train)):
    l= [x for x in X_train['new'].iloc[i]]
    l.append(X_train['Toss1'].iloc[i])
    l.append(X_train['Toss2'].iloc[i])
    l.append(X_train['Venue'].iloc[i])
    X_train['all'].iloc[i]=l


<ipython-input-31-69a0f875f029>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['all']=''
<ipython-input-31-69a0f875f029>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['all'].iloc[i]=l


In [32]:
l=[]
X_test['all']=''
for i in range(len(X_test)):
    l= [x for x in X_test['new'].iloc[i]]
    l.append(X_test['Toss1'].iloc[i])
    l.append(X_test['Toss2'].iloc[i])
    l.append(X_test['Venue'].iloc[i])
    X_test['all'].iloc[i]=l

<ipython-input-32-c9c5a48328bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['all']=''
<ipython-input-32-c9c5a48328bd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['all'].iloc[i]=l


### Model 

In [53]:
x_train=np.array(X_train['all'])
x_train=list(x_train)

In [54]:
x_test=np.array(X_test['all'])
x_test=list(x_test)

In [35]:
def model_eval(model, x_train, y_train, target, x_test,y_test):
    model.fit(x_train, y_train[target])
    predictions = model.predict(x_test)
    
    rms = sklearn.metrics.mean_squared_error(predictions,y_test[target])**0.5
    print('RMS : %s' % '{0:.2%}'.format(rms))
    
    r2 = sklearn.metrics.r2_score(predictions,y_test[target])
    print('R2 : %s' % '{0:.2%}'.format(r2))
    return np.asarray(predictions)

In [42]:
def winner_pred(model, X_train, Y_train, x_test, y_test):
    pred1 = model_eval(model, X_train, y_train,['Score_1'],x_test, y_test)
    pred2 = model_eval(model, X_train, y_train,['Score_2'],x_test, y_test)

    pred = pred1 - pred2

    for i in range(len(pred)):
        if ((pred[i])>0):
            pred[i] = 1
        else:
            pred[i] = 2

    #print(pred)
    sum=0
    print("Model Accuracy is: ")
    for i in range(len(pred)):
        if(pred[i]==y_test['Result'].iloc[i]):
            sum=sum+1
    print(sum/len(pred))
    

In [59]:
model = DecisionTreeRegressor()
winner_pred(model, x_train, y_train,x_test,y_test)

RMS : 6303.17%
R2 : -2079.97%
RMS : 5867.28%
R2 : -3022.45%
Model Accuracy is: 
0.5


In [55]:
model = LinearRegression()
winner_pred(model, x_train, y_train,x_test,y_test)

RMS : 6421.37%
R2 : -986.36%
RMS : 5573.20%
R2 : -1211.15%
Model Accuracy is: 
0.5


In [58]:
model = RandomForestRegressor()
winner_pred(model, x_train, y_train,x_test,y_test)

RMS : 4592.79%
R2 : -260316.05%
RMS : 4254.56%
R2 : -25386.79%
Model Accuracy is: 
0.5


<ipython-input-35-c2137f2e14d1>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train[target])
<ipython-input-35-c2137f2e14d1>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train[target])


In [60]:
x_train=np.array(X_train['new'])
x_train=list(x_train)

In [61]:
x_test=np.array(X_test['new'])
x_test=list(x_test)

In [69]:
model = DecisionTreeRegressor()
winner_pred(model, x_train, y_train,x_test,y_test)

RMS : 1614.00%
R2 : 0.00%
RMS : 5028.92%
R2 : 0.00%
Model Accuracy is: 
1.0


In [73]:
model = LinearRegression()
winner_pred(model, x_train, y_train,x_test,y_test)

RMS : 6421.41%
R2 : -986.39%
RMS : 5573.21%
R2 : -1211.17%
Model Accuracy is: 
0.5
